# Метод 3
## Использование линейных моделей

#### У нас уже есть словарь с названиями категорий и словами

In [2]:
import pandas as pd

In [3]:
data = pd.read_pickle("res.p")

In [4]:
for key in data:
    len([a for a in data[key] if a])

90

91

85

93

24

...

91

90

90

93

98

In [5]:
X, Y = [], []
k = 0
for key in data:
    value = data[key]
    for words in value:
        if words:
            X.append(" ".join(words))
            Y.append(key)
        else:
            k += 1

In [6]:
len(X), len(Y), k

(1024, 1024, 66)

### Снова сделаем из этого векторы и обучим модель
#### Сразу напишем Pipeline для этого

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

text_clf_1 = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier()),
])

text_clf_2 = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', RandomForestClassifier())
])

In [9]:
text_clf_1.fit(X_train, Y_train)
text_clf_2.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [10]:
predicted_1 = text_clf_1.predict(X_test)
predicted_2 = text_clf_2.predict(X_test)

In [11]:
import numpy as np

In [12]:
np.mean(predicted_1 == Y_test)
np.mean(predicted_2 == Y_test) 

0.71707317073170729

0.58536585365853655

# Да уж, что-то RandomForestClassifier не оправдал моих ожиданий на стандартных настройках. Оставим его и продолжим работать с SGDClassifier

In [13]:
predicted_1[36]

'Кино'

In [15]:
X_test[36][:300]

'поиск фильм персоналия видеорелиз релиз компания расширить поиск самый главный что новое каталог жанр год владелец лицензия выпустить премия награда коллекция ссылка видеобизнес релиз неделя архив лидер продажа итог год итог год итог год итог год обратить внимание релиз неделя архив лидер продажа ит'

## 72% просто стандартной конфигурацией...
### Запустим Grid Search, чтобы получить более лучшую конфигурацию

In [19]:
from sklearn.grid_search import GridSearchCV

# Я просто взял параметры отсюда http://scikit-learn.org/dev/auto_examples/model_selection/grid_search_text_feature_extraction.html
# но после первого применения изменил под себя
parameters = {
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2'),
    'classifier__loss': ('hinge', 'log', 'squared_hinge'),
    'classifier__alpha': (1e-03, 1e-07),
#     'classifier__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


In [20]:
grid_search = GridSearchCV(text_clf_1, parameters, n_jobs=-1, verbose=1)

In [21]:
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  1.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 1.0), 'classifier__alpha': (0.001, 1e-07), 'classifier__loss': ('hinge', 'log', 'squared_hinge'), 'vect__max_features': (None, 5000), 'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [22]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.751
Best parameters set:
	classifier__alpha: 0.001
	classifier__loss: 'log'
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__ngram_range: (1, 2)


In [21]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.5, max_features=5000, ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier(alpha=0.001, loss='log', penalty='l2')),
])

In [22]:
text_clf.fit(X_train, Y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == Y_test)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=5000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

0.73658536585365852

# Результаты разные. Дадим SGDClassifier'у второй шанс, увеличив и изменив обучающую выборку

In [ ]:
data = pd.read_pickle("res_2.p")

In [ ]:
for key in data:
    len([a for a in data[key] if a])

In [ ]:
X, Y = [], []
k = 0
for key in data:
    value = data[key]
    for words in value:
        if words:
            X.append(" ".join(words))
            Y.append(key)
        else:
            k += 1

In [ ]:
len(X), len(Y), k

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.5, max_features=5000, ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier(alpha=0.001, loss='log', penalty='l2')),
])

In [ ]:
text_clf.fit(X_train, Y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == Y_test)